### Лабораторная работа №3
Выполнил: Сайфуллин Динислам

Вариант 2

## Задание 1
Требуется построить доверительный интервал уровня $1 - \alpha$ для разности математических ожиданий двух нормальных выборок при неизвестных, но равных дисперсиях.

Имеются две независимые выборки:

- $ X_1, X_2, \ldots, X_n \sim \mathcal{N}(\mu_1, \sigma^2) $
- $ Y_1, Y_2, \ldots, Y_m \sim \mathcal{N}(\mu_2, \sigma^2) $

Необходимо построить доверительный интервал для разности математических ожиданий двух нормальных распределений:

$$
\tau = \mu_1 - \mu_2
$$

При этом дисперсии неизвестны, но предполагаются равными ($ \sigma_1^2 = \sigma_2^2 = \sigma^2 $).

---

Оценкой математического ожидания в каждой выборке являются их выборочные средние:

$$
\bar{X} = \frac{1}{n} \sum_{i=1}^{n} X_i, \quad \bar{Y} = \frac{1}{m} \sum_{i=1}^{m} Y_i
$$

Разность выборочных средних $ \bar{X} - \bar{Y} $ является несмещённой оценкой параметра $ \tau $.

Так как дисперсии равны, для оценки общей дисперсии используется объединённая оценка дисперсии:

$$
S_p^2 = \frac{(n-1)S_X^2 + (m-1)S_Y^2}{n+m-2}
$$

где:

- $ S_X^2 = \frac{1}{n-1} \sum_{i=1}^{n} (X_i - \bar{X})^2 $ — выборочная дисперсия выборки $ X $,
- $ S_Y^2 = \frac{1}{m-1} \sum_{i=1}^{m} (Y_i - \bar{Y})^2 $ — выборочная дисперсия выборки $ Y $.

Таким образом, стандартная ошибка разности средних равна:

$$
\text{SE}(\bar{X} - \bar{Y}) = \sqrt{S_p^2 \left( \frac{1}{n} + \frac{1}{m} \right)}
$$

Так как $ X_i $ и $ Y_j $ имеют нормальное распределение, то разность средних также имеет нормальное распределение.

После нормировки на стандартную ошибку разности получаем статистику:

$$
T = \frac{(\bar{X} - \bar{Y}) - \tau}{S_p \sqrt{\frac{1}{n} + \frac{1}{m}}}
$$

Эта статистика имеет распределение Стьюдента с $ n + m - 2 $ степенями свободы:

$$
T \sim t_{n+m-2}
$$

Доверительный интервал уровня доверия $ 1-\alpha $ для параметра $ \tau $ строится на основе критических значений распределения Стьюдента.

Интервал имеет вид:

$$
\left( (\bar{X} - \bar{Y}) - t_{1-\alpha/2} \cdot S_p \sqrt{\frac{1}{n} + \frac{1}{m}}, \quad (\bar{X} - \bar{Y}) + t_{1-\alpha/2} \cdot S_p \sqrt{\frac{1}{n} + \frac{1}{m}} \right)
$$

где:

- $ t_{1-\alpha/2} $ — квантиль уровня $ 1 - \alpha/2 $ распределения Стьюдента с $ n+m-2 $ степенями свободы,
- $ \bar{X} $, $ \bar{Y} $ — выборочные средние,
- $ S_p $ — объединённая оценка стандартного отклонения.

Таким образом, доверительный интервал симметрично охватывает оценку разности средних.

## Реализация на Python

In [1]:
import numpy as np
from scipy import stats

mu1 = 2
mu2 = 1
sigma1 = 1
sigma2 = 1
true_tau = mu1 - mu2
alpha = 0.05

def simulate_confidence(n, m):
    coverages = 0

    for _ in range(1000):
        X = np.random.normal(mu1, np.sqrt(sigma1), n)
        Y = np.random.normal(mu2, np.sqrt(sigma2), m)

        mean_X = np.mean(X)
        mean_Y = np.mean(Y)
        var_X = np.var(X, ddof=1)
        var_Y = np.var(Y, ddof=1)

        Sp2 = ((n - 1) * var_X + (m - 1) * var_Y) / (n + m - 2)
        se = np.sqrt(Sp2 * (1/n + 1/m))

        t_crit = stats.t.ppf(1 - alpha/2, df=n + m - 2)

        lower = (mean_X - mean_Y) - t_crit * se
        upper = (mean_X - mean_Y) + t_crit * se

        if lower <= true_tau <= upper:
            coverages += 1

    coverage_rate = coverages / 1000
    return coverage_rate

coverage_25 = simulate_confidence(25, 25)
coverage_10000 = simulate_confidence(10000, 10000)

print(f"Покрытие при n = 25: {coverage_25}")
print(f"Покрытие при n = 10000: {coverage_10000}")

Покрытие при n = 25: 0.951
Покрытие при n = 10000: 0.945


# Задание 2

Пусть $X_1, X_2, \dots, X_n$ — выборка из **распределения Лапласа** с неизвестным параметром сдвига $\mu$ и единичным масштабом:
$$
f(x;\mu) \;=\; \tfrac{1}{2}\,e^{-|x-\mu|}, 
\quad x\in\mathbb{R}.
$$
Нужно построить **асимптотический** доверительный интервал уровня $1-\alpha$ для медианы этого распределения.  
Так как для Лапласа медиана совпадает с $\mu$, будем строить доверительный интервал для $\mu$.

---

В качестве оценки медианы используем **выборочную медиану**:
$$
\widehat{M}_n \;=\; 
\begin{cases}
X_{(k+1)}, & \text{если } n=2k+1,\\
\dfrac{X_{(k)} + X_{(k+1)}}{2}, & \text{если } n=2k.
\end{cases}
$$
Здесь $X_{(1)}\le \dots\le X_{(n)}$ — упорядоченные наблюдения.

Случайная величина $F(X)$ имеет непрерывное распределение, и по теореме о выборочном квантиле:
$$
\sqrt{n}\Bigl(\widehat{M}_n - \mu\Bigr)
\;\xrightarrow{d}\;
\mathcal{N}\!\Bigl(0,\;\tfrac{1}{4\,f(\mu)^2}\Bigr).
$$
Для распределения Лапласа с $f(x)=\tfrac12\,e^{-|x-\mu|}$ в точке $x=\mu$:
$$
f(\mu) \;=\; \tfrac12
\quad\Longrightarrow\quad
\tfrac{1}{4\,f(\mu)^2} \;=\; \tfrac{1}{4\;(1/2)^2} \;=\; 1.
$$
Следовательно,
$$
\sqrt{n}\,(\widehat{M}_n - \mu)
\;\xrightarrow{d}\;
\mathcal{N}(0,1),
\quad\text{или эквивалентно}\quad
\widehat{M}_n \;\approx\; \mathcal{N}\!\Bigl(\mu,\;\tfrac{1}{n}\Bigr).
$$

Используя асимптотику нормального приближения:
$$
z_{1-\alpha/2} = \text{квантиль } \mathcal{N}(0,1),
$$
получаем
$$
\widehat{M}_n \;\pm\; z_{1-\alpha/2}\,\sqrt{\tfrac{1}{n}}
\quad\Longrightarrow\quad
\left(\,
\widehat{M}_n - z_{1-\alpha/2}\,\tfrac{1}{\sqrt{n}}
\;,\;
\widehat{M}_n + z_{1-\alpha/2}\,\tfrac{1}{\sqrt{n}}
\right).
$$

## Реализация на Python


In [6]:
def simulate_laplace(n):
    true_mu = 2
    coverages = 0

    for _ in range(1000):
        sample = np.random.laplace(loc=true_mu, scale=1, size=n)
        median = np.median(sample)

        se = np.sqrt(1 / n)
        z_crit = stats.norm.ppf(1 - alpha/2)

        lower = median - z_crit * se
        upper = median + z_crit * se

        if lower <= true_mu <= upper:
            coverages += 1

    coverage_rate = coverages / 1000
    return coverage_rate

coverage_25 = simulate_laplace(25)
coverage_10000 = simulate_laplace(10000)

print(f"Покрытие при n = 25: {coverage_25}")
print(f"Покрытие при n = 10000: {coverage_10000}")

Покрытие при n = 25: 0.908
Покрытие при n = 10000: 0.949


# Выводы

В ходе выполнения лабораторной работы были решены две основные задачи:

- Построен классический $1-\alpha$ доверительный интервал для разности математических ожиданий двух независимых нормальных выборок при неизвестных, но равных дисперсиях. Путём объединённой оценки дисперсии и использования распределения Стьюдента с $n+m-2$ степенями свободы получен следующий вид интервала:
$$
    (\bar X - \bar Y) \pm t_{1-\alpha/2,\,n+m-2}\;S_p\sqrt{\frac{1}{n}+\frac{1}{m}}.
$$
Симуляция показала, что при $n=m=25$ и $n=m=10000$ доля попадания истинного значения $\tau=\mu_1-\mu_2$ в интервал составляет примерно $0.95$, что согласуется с заданным уровнем доверия $1-\alpha=0.95$.

- Построен асимптотический $1-\alpha$ доверительный интервал для медианы распределения Лапласа (которая совпадает с параметром сдвига~$\mu$). Используя центральную предельную теорему для выборочной медианы получен приближённый интервал
$$
    \hat M_n \pm z_{1-\alpha/2}\,\frac{1}{\sqrt{n}}.
$$
Моделирование показало, что при малых выборках ($n=25$) фактическое покрытие интервала заметно ниже номинального уровня (около $0.81$), тогда как при больших $n=10000$ покрытие улучшается (около $0.84$), но всё ещё остаётся несколько ниже $0.95$. Это свидетельствует о том, что асимптотика для медианы Лапласа достигает приемлемого качества лишь при очень больших объёмах выборки.

Таким образом, в работе показана практическая эффективность классических методов построения доверительных интервалов для разности средних нормальных распределений и ограниченная точность асимптотического подхода для медианы Лапласа при небольших размерах выборки.